In [1]:
from numpy import pi, linspace, diff, mean, min, max, arctan2, diff
from skyfield.api import load
from skyfield.framelib import ecliptic_J2000_frame
from skyfield.searchlib import find_discrete
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearAxis, Range1d

output_notebook()

Loading BokehJS ...

In [2]:
ts = load.timescale()
# eph_name = 'de421.bsp'
eph_name = 'de422.bsp'
t_start = ts.tt(1750, 6, 21, 0, 0, 0)
planets = ["mercury", "venus", "earth", "moon", "mars", "jupiter_barycenter", "saturn_barycenter", "uranus_barycenter", "neptune_barycenter"]
eph = load(eph_name)

## Orbit angle to the elliptical

In [3]:
def get_inclination_ecliptic(obj, ref, t_start, years):
    def f_vel():
        def zero(t):
            _, vel = (obj.at(t) - ref.at(t)).frame_xyz_and_velocity(ecliptic_J2000_frame)
            return vel.km_per_s[2] > 0
        zero.step_days = 10.0
        return zero
    
    zero_vel = find_discrete(t_start, t_start + 365.25 * years, f_vel())
    t_high = zero_vel[0][zero_vel[1] == 0]
    lat, _, _ = (obj.at(t_high) - ref.at(t_high)).frame_latlon(ecliptic_J2000_frame)
    return t_high, lat

In [4]:
for p in planets:
    if p == "earth":
        continue
    ref = eph["sun"]
    years = 400
    if p.lower() == "moon":
        ref = eph["earth"]
        years = 20
    obj = eph[p]
    t_high, lat = get_inclination_ecliptic(obj, ref, t_start, years)
    colors = [f"#{255:02x}{int(255 * (t_high.tt[-1] - t) / (t_high.tt[-1] - t_high.tt[0])):02x}{255:02x}" for t in t_high.tt]
    fig = figure(title=p.capitalize() + ", orbit angle to ecliptic, starting at " + t_start.tt_strftime(), x_axis_label="years", y_axis_label="degrees", width=1100)
    fig.scatter((t_high.tt - t_high.tt[0])/365.25, lat.degrees , fill_color=colors, line_color="red", size = 5)
    fig.line((t_high.tt - t_high.tt[0])/365.25, lat.degrees, line_color="blue")

    show(fig)

Good agreement with oficial values, except for Mercury, which is slightly off.

## Perihelion, Aphelion

In [5]:
def get_distances(obj, ref, t_start, years):
    def f_vel():
        def zero(t):
            _, _, _, _, _, radial_vel = (obj.at(t) - ref.at(t)).frame_latlon_and_rates(ecliptic_J2000_frame)
            return radial_vel.km_per_s > 0
        zero.step_days = 10.0
        return zero
    
    zero_vel = find_discrete(t_start, t_start + 365.25 * years, f_vel())
    t_min = zero_vel[0][zero_vel[1] == 1]
    _, _, d_min = (obj.at(t_min) - ref.at(t_min)).frame_latlon(ecliptic_J2000_frame)
    t_max = zero_vel[0][zero_vel[1] == 0]
    _, _, d_max = (obj.at(t_max) - ref.at(t_max)).frame_latlon(ecliptic_J2000_frame)
    return t_min, d_min.au, t_max, d_max.au

In [6]:
perihelions = []
aphelions = []
for p in planets:
    ref = eph["sun"]
    title = ", Perihelion and Aphelion"
    years = 400
    if p.lower() == "moon":
        ref = eph["earth"]
        years = 20
        title = ", Perigee and Apogee"
    obj = eph[p]
    t_min, d_min, t_max, d_max = get_distances(eph[p], ref, t_start, years)
    peri = sum(d_min)/len(d_min)
    aph = sum(d_max)/len(d_max)
    perihelions += [peri]
    aphelions += [aph]
    
    fig = figure(title=p.capitalize() + title + ", starting at " + t_start.tt_strftime(), x_axis_label="years", y_axis_label="AU", width=1100)
    fig.y_range = Range1d(start=min(d_min), end=max(d_min))
    fig.extra_y_ranges = {"r": Range1d(start=min(d_max), end=max(d_max))}
    fig.add_layout(LinearAxis(y_range_name="r"), 'right')
    
    fig.scatter((t_min.tt - t_min.tt[0])/365.25, d_min, color="red", size=5, legend_label="Perihelion (left)")
    fig.scatter((t_max.tt - t_max.tt[0])/365.25, d_max, color="blue", size=5, legend_label="Aphelion (right)", y_range_name="r")  
    fig.line((t_min.tt - t_min.tt[0])/365.25, peri, line_color="red", line_dash=[4, 4], legend_label="Perihelion average")
    fig.line((t_max.tt - t_max.tt[0])/365.25, aph, line_color="blue", line_dash=[4, 2], legend_label="Aphelion average", y_range_name="r")
    
    show(fig)

In [7]:
for i in range(len(planets)):
    print(planets[i], perihelions[i], aphelions[i])

mercury 0.3075026910367949 0.4666944466056256
venus 0.7184160795272655 0.7282470672611944
earth 0.9832693264156529 1.016731774265009
moon 0.0024239188762744434 0.0027099362528621527
mars 1.3814410141716325 1.6659354167972746
jupiter_barycenter 4.95067876834778 5.455074274498264
saturn_barycenter 9.019651397243326 10.061885076371121
uranus_barycenter 18.28460944567474 20.096475885496723
neptune_barycenter 30.032522397012283 30.201523952928017


Good agreement with official numbers except for Neptune

## Orbit Sidereal Period

In [8]:
def get_lon_times(obj, ref, t_start, years):
    def f_():
        def longitude(t):
            _, lon, _ = (obj.at(t) - ref.at(t)).frame_latlon(ecliptic_J2000_frame)
            return lon.degrees > 180
        longitude.step_days = 10.0
        return longitude
    
    zero = find_discrete(t_start, t_start + 365.25 * years, f_())
    t_ = zero[0][zero[1] == 1]
    return t_

In [10]:
for p in planets:
    ref = eph["sun"]
    years = 400
    if p.lower() == "moon":
        ref = eph["earth"]
        years = 20
    obj = eph[p]
    times = get_lon_times(obj, ref, t_start, years)
    fig = figure(title=p.capitalize() + ", Sidereal Period, starting at " + times[1].tt_strftime(), x_axis_label="years", y_axis_label="days", width=1100)
    fig.scatter((times.tt[1:] - times.tt[0])/365.25, diff(times.tt), line_color="red", size = 5)
    fig.line((times.tt[1:] - times.tt[0])/365.25, diff(times.tt), line_color="blue")
    show(fig)